In [14]:
from pathlib import Path
import json

In [15]:
asr_data = json.load(Path("李昆澤-127844.gasr.json").open(encoding="UTF-8"))
metadata = json.load(Path("metadata.json").open(encoding="UTF-8"))

In [16]:
asr = asr_data["response"]["results"]
transcript = [x for x in metadata if x["video_id"]=="127844"][0]["transcript"]

## 議事錄逐字稿

In [17]:
import re
from itertools import tee
def preprocess_transcript(txt):
    txt = txt.replace("<br />", "")
    txt = re.split("(\n?.+?：)", txt)    
    txt = [x.replace("\n", "").strip() for x in txt]
    txt = [x for x in txt if x]
    iter_list = [iter(txt)] * 2
    turns = [(a,b) for a,b in zip(*iter_list)]
    return turns

In [19]:
turns = preprocess_transcript(transcript)
turns[:5]

[('李委員昆澤：',
  '（10 時 13 分）部長，您好。遊艇證照弊案傷害政府的威信，也影響到考照的公平性，更是衝擊到航運的安全，這一次的事件整體來看，內部監理出了重大漏洞，這個部分請部長簡單說明一下。'),
 ('主席：', '請交通部林部長說明。'),
 ('林部長佳龍：',
  '不管原因是什麼，我們要阻止類似的事情再發生，所以我在第一時間得知檢調在偵辦，就要求同仁同步清查並全面提供資料，而且要澈底建立防弊的措施，不管是相關的作業程序或者承辦人的職權，整個檢討跟改進。'),
 ('李委員昆澤：',
  '航港局的檢討報告提到，要限縮相關人員的權限，加強勾稽比對，並由科長來核對、覆核，系統要更新，限制成績須整批匯入，發照人員也要定期輪調。我也具體的建議，我們必須要加強內部稽核和不定期查核，請簡單說明。'),
 ('主席：', '請交通部航港局葉局長說明。')]

## ASR

In [6]:
from itertools import chain

def to_num(x):
    return float(x.replace("s", ""))

def flatten_asr(asr_obj):    
    asr_starts = []
    asr_ends = []
    asr_words = []
    for entry_x in asr_obj:
        alts = entry_x.get("alternatives", [])
        if not alts: continue
        words = alts[0].get("words", [])        
        asr_starts.extend([
            to_num(x["startTime"]) for x in words])
        asr_ends.extend([
            to_num(x["endTime"]) for x in words])
        asr_words.extend([x["word"] for x in words])
        
    return {
        "starts": asr_starts,
        "ends": asr_ends,
        "words": asr_words
    }

flat_asr = flatten_asr(asr)
assert len(flat_asr["starts"]) == len(flat_asr["words"])

In [23]:
asr[1]

{'alternatives': [{'confidence': 0.923182,
   'transcript': ' 不長你好，遊艇證照B按他傷害政府的微信也影響到口罩的公平性更是從啟導航運的安全，那這一次的事件整體來看內部的間裡啊，除了重大的漏洞，這個部分來請部長簡單說明一下，我不管他的原因是什麼，然後我們要阻止這個類似的事情再發生，那所以我在第一時間而得知檢調偵辦的時候，我就要求同仁同步的去青茶，然後全面提供資料，而且要把這一個房地的措施把他徹底建立，我不管是說這一些付把相關的這個作業程序法會的是承辦人的職權保證，各做一個檢討更改進這個。',
   'words': [{'endTime': '15.800s', 'startTime': '15.400s', 'word': '不'},
    {'endTime': '15.900s', 'startTime': '15.800s', 'word': '長'},
    {'endTime': '16s', 'startTime': '15.900s', 'word': '你'},
    {'endTime': '16.200s', 'startTime': '16s', 'word': '好'},
    {'endTime': '17.500s', 'startTime': '16.200s', 'word': '遊'},
    {'endTime': '17.700s', 'startTime': '17.500s', 'word': '艇'},
    {'endTime': '18s', 'startTime': '17.700s', 'word': '證'},
    {'endTime': '18.100s', 'startTime': '18s', 'word': '照'},
    {'endTime': '18.300s', 'startTime': '18.100s', 'word': 'b'},
    {'endTime': '18.600s', 'startTime': '18.300s', 'word': '按'},
    {'endTime': '19.200s', 'startTime': '18.600s', 'word': '他'},
    {'endTim

## 兩邊對齊

In [11]:
from difflib import SequenceMatcher
from tqdm.auto import tqdm
import numpy as np
win = 20
offset = 2
asr_words = flat_asr["words"]
asr_starts = flat_asr["starts"]
is_punct = lambda x: x in "，。：）（"

aligned_turn_start = []
for turn_x in tqdm(turns):
    probe = turn_x[1][offset:offset+win]    

    scores = []
    for i in range(len(asr_words)-win):                
        target = "".join(asr_words[i:i+win])
        sm = SequenceMatcher(is_punct,
                probe, target)
        scores.append(sm.ratio())

    align_idx = np.argmax(scores)
    print(f"[{max(scores)}]")
    print("Probe:", probe)
    print("Target: ", "".join(asr_words[align_idx:align_idx+20]))
    aligned_turn_start.append(asr_starts[align_idx])

  0%|          | 0/22 [00:00<?, ?it/s]

[0.3]
Probe: 0 時 13 分）部長，您好。遊艇證照弊
Target:  人不算機業局長有沒有局長不長你好遊艇證照
[0.35714285714285715]
Probe: 通部林部長說明。
Target:  啊除了重大的漏洞這個部分來請部長簡單說明
[0.8]
Probe: 原因是什麼，我們要阻止類似的事情再發生，
Target:  原因是什麼然後我們要阻止這個類似的事情再
[0.8]
Probe: 局的檢討報告提到，要限縮相關人員的權限，
Target:  局相關的檢討報告提到要先說相關人員的權限
[0.3225806451612903]
Probe: 通部航港局葉局長說明。
Target:  重大的一個影響那這個部分來警局長簡單說明
[0.65]
Probe: 委員，目前我們已經全面變更並改正系統的設
Target:  目前我們已經全面的這個變更這個改正這個系
[0.65]
Probe: 這個部分，剛才很多委員提到相關的題庫艱深
Target:  查核好這個部分啊我們剛才很多也提到相關的
[0.6]
Probe: 員報告，全球海運缺櫃的現象是受到新冠肺炎
Target:  部分全球的海運衰退現像是受到這個新冠肺炎
[0.7]
Probe: ，我們針對缺櫃這樣一個嚴重的狀況，有沒有
Target:  是局長我們針對缺櫃的嚴重的狀況我們有沒有
[0.65]
Probe: 員報告，去年 10 月部長就提醒我們有這
Target:  跟我們報告去年這個時樂部長就提醒我們有這
[0.6]
Probe: ，這個非常重要，因為我們在 1 月的時候
Target:  不然這個非常重要因為我們在醫院收看整體的
[0.65]
Probe: 現在大概不缺櫃而是價格的問題，至於供需，
Target:  數量的狀況又現在大概不缺櫃還是價格的問題
[0.7]
Probe: 必須要清楚掌握這些艙位以及貨櫃數量，因為
Target:  必需要清楚地掌握著香味以及貨櫃的數量因為
[0.0]
Probe: 
Target:  謝謝主席請夏林佳龍部長還很有錢人家人不算
[0.65]
Probe: 我要請教葉局長，有關於郵輪觀光的發展，不
Target:  在我請假謝其長在有關於郵輪觀光的一個發展
[0.9]
Probe: 員報告，這個部分的確跟原來預估有一個落差
Target:  印報

In [10]:
from datetime import timedelta
for start_time, (speaker, transcript) in zip(aligned_turn_start, turns):
    timestamp = str(timedelta(seconds=start_time)).split(".")[0]
    print("[{}] {} {}".format(timestamp, speaker, transcript))

[0:00:09] 李委員昆澤： （10 時 13 分）部長，您好。遊艇證照弊案傷害政府的威信，也影響到考照的公平性，更是衝擊到航運的安全，這一次的事件整體來看，內部監理出了重大漏洞，這個部分請部長簡單說明一下。
[0:00:34] 主席： 請交通部林部長說明。
[0:00:42] 林部長佳龍： 不管原因是什麼，我們要阻止類似的事情再發生，所以我在第一時間得知檢調在偵辦，就要求同仁同步清查並全面提供資料，而且要澈底建立防弊的措施，不管是相關的作業程序或者承辦人的職權，整個檢討跟改進。
[0:01:19] 李委員昆澤： 航港局的檢討報告提到，要限縮相關人員的權限，加強勾稽比對，並由科長來核對、覆核，系統要更新，限制成績須整批匯入，發照人員也要定期輪調。我也具體的建議，我們必須要加強內部稽核和不定期查核，請簡單說明。
[0:03:48] 主席： 請交通部航港局葉局長說明。
[0:01:43] 葉局長協隆： 謝謝委員，目前我們已經全面變更並改正系統的設定，還有受理申辦的流程，現在承辦人是沒有辦法做任何資料的更改，在發證前，系統也設定成績必須要合格才能夠發證，在考駕照的控管上，也由科長來控管空白駕照以及流水號，每天都必須要逐日核對，並按月做成紀錄往上呈報並做覆核。另外委員剛剛提醒的，要做實地和外部的稽核部分，我們去年 10 月和去年 12 月已經做了兩次到中部航務中心的外部稽核，今年 6 月和 7 月也分別會針對六大項目來進行進一步的實地查核。
[0:02:40] 李委員昆澤： 好，這個部分，剛才很多委員提到相關的題庫艱深，而且大部分都是船機的相關知識，我們應該強化考照的流程，另外針對題庫的合理性以及相關的適用性，必須提出更具體的檢討和改進，而且航行的安全教育真的是非常重要，也必須要加強。我現在要請教的是針對貨櫃的數量，航港局有沒有掌握？現在因為疫情的影響，全世界的海運快速發展，大家需求的貨櫃數量都大幅增加，甚至有錢可能也買不到貨櫃，相關的狀況，我們看到臺灣是一個重要的進出口國家，海運是我們重要的經濟支柱，相對的，海運如果缺乏貨櫃的話，恐怕對經濟會造成重大的影響，這部分請局長簡單說明一下。
[0:03:56] 葉局長協隆： 跟委員報告，全球海運缺櫃的現象是受到新冠肺炎疫情的影響，造成港口塞港以致貨櫃的回流速度減緩，造成整個缺櫃的情形。
[0:04:11] 李委員昆澤： 局長